In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import os
import matplotlib.pyplot as plt
import pandas as pd
import random
from tqdm import tqdm
import shutil
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import zipfile
import gzip, pickle
import xml.etree.ElementTree as ET 
from PIL import Image
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()


# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('../input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
with zipfile.ZipFile("../input/generative-dog-images/all-dogs.zip","r") as z:
    z.extractall(".")

In [ ]:
with zipfile.ZipFile("../input/generative-dog-images/Annotation.zip","r") as z:
    z.extractall(".")

In [ ]:
!ls .

In [ ]:
dogs = os.listdir('./all-dogs/')

In [ ]:
print("Total Images =", len(dogs))
print("Total breeds=", len(os.listdir('./Annotation')))

In [ ]:
All_dogs = os.listdir('./all-dogs/')
All_breeds = os.listdir('./Annotation/') 

counter = 0; names = []
Training_dogs = np.zeros((25000,64,64,3))

for breed in All_breeds:
    for dog in os.listdir('./Annotation/'+breed):
        try: img = Image.open('./all-dogs/'+dog+'.jpg') 
        except: continue           
        tree = ET.parse('./Annotation/'+breed+'/'+dog)
        root = tree.getroot()
        objects = root.findall('object')
        for o in objects:
            bndbox = o.find('bndbox') 
            xmin = int(bndbox.find('xmin').text)
            ymin = int(bndbox.find('ymin').text)
            xmax = int(bndbox.find('xmax').text)
            ymax = int(bndbox.find('ymax').text)
            w = np.min((xmax - xmin, ymax - ymin))
            img2 = img.crop((xmin, ymin, xmin+w, ymin+w))
            img2 = img2.resize((64,64), Image.ANTIALIAS)
            Training_dogs[counter,:,:,:] = np.asarray(img2)
            names.append(breed)
            counter += 1
idx = np.arange(counter)
np.random.shuffle(idx)
Training_dogs = Training_dogs[idx,:,:,:]
names = np.array(names)[idx]
    
x = np.random.randint(0,counter,25)
for k in range(5):
    plt.figure(figsize=(15,3))
    for j in range(5):
        plt.subplot(1,5,j+1)
        img = Image.fromarray( Training_dogs[x[k*5+j],:,:,:].astype('uint8') )
        plt.axis('off')
        plt.title(names[x[k*5+j]].split('-')[1],fontsize=11)
        plt.imshow(img)
    plt.show()


In [ ]:
print(Training_dogs.shape)
print(len(idx))

In [ ]:
img=Image.fromarray(Training_dogs[12121,:,:,:].astype('uint8') )
plt.imshow(img)

In [ ]:
def generate_mask(kernel_size, channels_in, channels_out, input_channels, mask_type, factorized=True):
    mask = np.zeros(shape=(kernel_size, kernel_size, channels_in, channels_out), dtype=np.float32)
    mask[:kernel_size//2, :, :, :] = 1
    mask[kernel_size//2, :kernel_size//2, :, :] = 1
    
    if factorized:
        if mask_type=='B':
            mask[kernel_size//2, kernel_size//2, :, :] = 1
    else:
        w = int(np.ceil(channels_out/input_channels))
        h = int(np.ceil(channels_in/input_channels))
        k = mask_type=='A'
        m0 = np.triu(np.ones(dtype=np.float32, shape=(input_channels,input_channels)), k)
        m1 = np.repeat(m0, w, axis=1)
        m2 = np.repeat(m1, h, axis=0)
        masked_pixel = m2[:channels_in,:channels_out]
        mask[kernel_size//2, kernel_size//2, :, :] = masked_pixel
        
    return mask

In [ ]:
def masked_conv2d(x, channels_out, kernel_size, input_channels, mask_type, factorized):
    # Get dimensions of the input tensor
    _, h, w, channels_in = x.shape.as_list()
    # Create weight and bias variables
    weights = tf.get_variable('weight', shape=(kernel_size, kernel_size, channels_in, channels_out),  trainable=True)
    bias = tf.get_variable('bias', shape=(h, w, channels_out), trainable=True)
    # Create the mask
    mask = generate_mask(kernel_size, channels_in, channels_out, input_channels=input_channels, mask_type=mask_type, factorized=factorized)
    # Apply convolution
    y = tf.nn.conv2d(x, weights*mask, strides=[1, 1, 1, 1], padding='SAME') + bias
    return y

In [ ]:
def residual_block(x_in, channels_out=128, input_channels=None, factorized=True):
    x = tf.nn.relu(x_in)
    # Downsample channels using 1x1 convolution
    with tf.variable_scope('downsample'):
        x = masked_conv2d(x, channels_out=channels_out, kernel_size=1, input_channels=input_channels, mask_type='B', factorized=factorized)
    x = tf.nn.relu(x)
    # Main convolution
    with tf.variable_scope('conv'):
        x = masked_conv2d(x, channels_out=channels_out, kernel_size=3, input_channels=input_channels, mask_type='B', factorized=factorized)
    x = tf.nn.relu(x)
    # Upsample channels by two using 1x1 convolution
    with tf.variable_scope('upsample'):
        x = masked_conv2d(x, channels_out=channels_out*2, kernel_size=1, input_channels=input_channels, mask_type='B', factorized=factorized)
    return x + x_in

In [ ]:
def pixel_cnn(x, channels_out, factorized):
    input_channels = x.shape.as_list()[3]
    inp = tf.cast(x, tf.int32)
    x = tf.cast(x, tf.float32)
    with tf.variable_scope('input_conv'):
        x = masked_conv2d(x, channels_out=128*2, kernel_size=7, input_channels=input_channels, mask_type='A', factorized=factorized)
    for i in range(12):
        with tf.variable_scope('res_block_%d' % i):
            x = residual_block(x, channels_out=128, input_channels=input_channels, factorized=factorized)
    with tf.variable_scope('output_conv_1'):
        x = tf.nn.relu(x)
        x = masked_conv2d(x, channels_out=128, kernel_size=1, input_channels=input_channels, mask_type='B', factorized=factorized)
    with tf.variable_scope('output_conv_2'):
        x = tf.nn.relu(x)
        x = masked_conv2d(x, channels_out=channels_out, kernel_size=1, input_channels=input_channels, mask_type='B', factorized=factorized)
    
    x_rshp = tf.reshape(x, [-1, 64, 64, 3, 256])
    losses = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=inp, logits=x_rshp)
    loss = tf.reduce_mean(losses) * np.log2(np.e)
    probs = tf.nn.softmax(x_rshp)
    return loss, probs, x

In [ ]:
def create_dataset(x, batch_size):
    dataset = tf.data.Dataset.from_tensor_slices(x)
    dataset = dataset.repeat()   # Repeat the dataset indefinitely
    dataset = dataset.shuffle(10000)   # Shuffle the data
    dataset = dataset.batch(batch_size)  # Create batches of data
    dataset = dataset.prefetch(batch_size)  # Prefetch data for faster consumption
    iterator = tf.data.make_initializable_iterator(dataset)  # Create an iterator over the dataset
    return iterator

In [ ]:

nrof_epochs = 10
batch_size = 32
factorized = True

tf.reset_default_graph()
with tf.Graph().as_default():

        train_iterator=create_dataset(Training_dogs[:16000, :, :, :], batch_size)
        eval_sample = tf.placeholder(tf.float32, shape=(None, 64, 64, 3))
        
        with tf.variable_scope('model', reuse=False):
            train_loss, _, _ = pixel_cnn(train_iterator.get_next(), channels_out=3*256, factorized=factorized)
            
        with tf.variable_scope('model', reuse=True):
            eval_sample_loss, eval_sample_probs, _ = pixel_cnn(eval_sample, channels_out=3*256, factorized=factorized)
            
    
        optimizer = tf.train.AdamOptimizer(learning_rate=0.001)
        tvars = tf.trainable_variables()
        grads, _ = tf.clip_by_global_norm(tf.gradients(train_loss, tvars), 1.0)
        train_op = optimizer.apply_gradients(zip(grads, tvars))

        sess  = tf.InteractiveSession()
        sess.run(tf.global_variables_initializer())
        sess.run(train_iterator.initializer)
   

        nrof_train_batches = 16000//batch_size

        train_loss_list = []
        for epoch in range(1, nrof_epochs+1):
            for i in range(nrof_train_batches):
                _, loss_ = sess.run([train_op, train_loss])
                train_loss_list += [ loss_ ]
                if i % 25 == 0:
                    print('train epoch: %4d  batch: %4d  loss: %7.3f' % (epoch, i, loss_))

    

In [ ]:
def display_samples(samples):
      for k in range(4):
        plt.figure(figsize=(16,4))
        for j in range(4):
            plt.subplot(1,4,j+1)
            img = Image.fromarray( samples[k*4+j,:,:,:].astype('uint8') )
            plt.axis('off')
            plt.imshow(img)
        plt.show()

In [ ]:
def sample(No_of_samples, noise=None):
    img = np.zeros((No_of_samples, 64, 64, 3), dtype=np.uint8)
    img[:, 0, 0, 0] = np.random.choice(256, size=(No_of_samples,))
    for j in range(64):
        for k in range(64):
            for l in range(3):
                loss, p = sess.run([eval_sample_loss, eval_sample_probs], feed_dict={eval_sample:img})
                for i in range(No_of_samples):
                    img[i, j, k, l] = np.random.choice(256, p=p[i, j, k, l])
    return img

In [ ]:
images=sample(16)

In [ ]:
plt.figure()
display_samples(images)